In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.pyplot import MultipleLocator
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix

In [ ]:
data = pd.read_excel('V8_RFE6_ranked.xlsx')
data.head()

In [ ]:
data2 = data.drop(labels=['Unnamed: 0','序号'],axis=1)
Importance_corr = data2.corr()['Dimensionality'].abs().sort_values(ascending=False)
Importance_corr.to_excel('2_XYPlsenImp200.xlsx')

In [ ]:
data3 = pd.read_excel('V4_XYPlsImp200.xlsx')
data3.head()

In [ ]:
X = data3.iloc[:,3:]
Y = data3.iloc[:,2:3]
X.shape

In [ ]:
#剔除掉相关性大于某一值的所有特征；
X_corr2 = round(X.corr().abs(),2)
correlated_features = set()  #满足某一相关性的高相关性特征集

for i in range(len(X_corr2.columns)):
    for j in range(i):
        if abs(X_corr2.iloc[i,j])>0.8:
            colname=X_corr2.columns[i]
            correlated_features.add(colname)

X.drop(labels=correlated_features,axis=1,inplace=True)  #剔除掉这些高相关性特征
X.shape

In [ ]:
X.head()

In [ ]:
X.to_excel('X.xlsx')

In [ ]:
data4 = pd.concat([data3.iloc[:,1:3],X],axis=1)
#data_Xnml = pd.DataFrame(X_nml)
data4.to_excel('V12_below08_.xlsx')

In [ ]:
data5 = pd.read_excel('V8_RFE6_ranked.xlsx')
data5.head()

In [ ]:
X = data5.iloc[:,3:]
y = data5.iloc[:,2:3]
X.head()

In [ ]:
#前41个特征的相关性热图
X_corr = round(X.corr().abs(),2)

plt.figure(dpi=200)
sns.set(font_scale=1.2)
plt.rc('font',family='Arial')
sns.heatmap(X_corr,cmap="GnBu",
            annot=True,annot_kws={'size':14},
           vmax=1,
           linewidths=1)
plt.xticks(rotation=90)
plt.savefig('Figure7_heatmapPersen6.png',dpi=2160,bbox_inches='tight')
#plt.show()

In [ ]:
# 进行RFE（Recursive feature elimination）计算-RF
# https://zhuanlan.zhihu.com/p/144847932
RF = RandomForestClassifier()
rfe = RFE(estimator=RF, n_features_to_select=1, step=1)
rfe.fit(X,y)
print(rfe.n_features_)  # 打印最优特征变量数
print(rfe.support_)  # 打印选择的最优特征变量
print(rfe.ranking_)  # 特征排序

In [ ]:
rfe_DF = pd.DataFrame(rfe.ranking_)
rfe_DF.to_excel('Figure3_RFEranking10.xlsx')

In [ ]:
data_rfecv = pd.read_excel('V7_RFE6.xlsx')
data_rfecv.head()

In [ ]:
X = data_rfecv.iloc[:,3:]
y = data_rfecv.iloc[:,2:3]
X.head()

In [ ]:
# 进行RFE（Recursive feature elimination）计算-RF
# https://zhuanlan.zhihu.com/p/144847932
RF = RandomForestClassifier()
rfe = RFE(estimator=RF, n_features_to_select=1, step=1)
rfe.fit(X,y)
print(rfe.n_features_)  # 打印最优特征变量数
print(rfe.support_)  # 打印选择的最优特征变量
print(rfe.ranking_)  # 特征排序

In [ ]:
c = pd.DataFrame(rfe.ranking_)
c.to_excel('Figure6_RFE6ranking.xlsx')

In [ ]:
# RFEcv_RF分类器
RF = RandomForestClassifier()
rfecv = RFECV(estimator=RF, step=1, cv=StratifiedKFold(2),scoring='accuracy')
rfecv.fit(X, y)
print("RFEC挑选了几个特征 : %d" % rfecv.n_features_)
print("重要性排名：",rfecv.ranking_)
print("最优特征变量:",rfecv.support_)  # 打印选择的最优特征变量
print("rfecv.grid_scores：",rfecv.grid_scores_)

plt.figure(dpi=800)
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1,len(rfecv.grid_scores_)+1), rfecv.grid_scores_)
plt.show()

In [ ]:
a = pd.DataFrame(rfecv.ranking_)
a.to_excel('Figure4_RFEcv_ranking10.xlsx')

In [ ]:
b = pd.DataFrame(rfecv.grid_scores_)
b.to_excel('Figure5_RFEcv_grid_scores10.xlsx')

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer
from sklearn.model_selection import train_test_split,KFold,LeaveOneOut,cross_val_score,cross_validate,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix

import warnings
warnings.filterwarnings("ignore")

In [ ]:
model_KNN = KNeighborsClassifier()
model_LR = LogisticRegression()
model_DT = DecisionTreeClassifier()
model_SVM = SVC()

model_MLP = MLPClassifier()
model_SGD = SGDClassifier()
model_RF = RandomForestClassifier()
model_AB = AdaBoostClassifier()
model_GB = GradientBoostingClassifier()

In [ ]:
data6 = pd.read_excel('V8_RFE6_ranked.xlsx')
data6.head()

In [ ]:
X = data6.iloc[:,3:6]
Y = data6.iloc[:,2:3]
X

In [ ]:
X_std = StandardScaler().fit_transform(X)
X_nml = Normalizer().fit_transform(X_std)

In [ ]:
#这里边只有准确率可以用于多分类，其他三个只能用于二分类，所有其他三个没有参考价值
KF_result = pd.DataFrame()
model_list = [model_KNN,model_LR,model_DT,model_SVM,model_MLP,model_SGD,model_RF,model_AB,model_GB]

for i,x in enumerate(model_list):
    scores1= cross_val_score(x,X_nml,Y,cv=10,scoring='accuracy')
    KF_result.loc[i,'accuracy_score'] = np.mean(scores1)
KF_result.index = pd.Series(model_list)
KF_result

In [ ]:
KF_result.to_excel('Table1_Kfold_crosscv.xlsx')

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=10)

In [ ]:
# model_RF调参
params = {'n_estimators':[2,5,10,20,50],
          'criterion':['gini','entropy'],
          'max_depth':[None,1,5,10,20],
          'min_samples_split':[1,2,5,10,20],
          'min_samples_leaf':[1,2,3,4,5,10,20]
         }
clf = GridSearchCV(model_RF,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_RF_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('a.xlsx')


In [ ]:
best_RF = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_RF_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_RF.loc[i,'acc_score_RF'] = acc_score
    #print(acc_score)
best_RF

In [ ]:
# model_GB调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
# 调参：https://blog.csdn.net/VariableX/article/details/107200334
params = {'loss': ['deviance'], 
          'learning_rate': [0.01],
          #'learning_rate': [0.001,0.01,0.1,1],
          'subsample': [0.7], 
          #'subsample': [0.5,0.6,0.7,0.8,0.9,1,2,5,10],
          'n_estimators': [10,50,100,120,300,500,800,1200], 
          #'n_estimators': [300],
         # 'criterion': ['entropy','gini'],
          #'max_depth': [1,2,3,5,8,15,None],
          'max_depth': [8],
          #'min_samples_leaf':[1,2,5,10],
          'min_samples_leaf':[10],
          #'min_samples_split':[2,5,10,20,50],
          'min_samples_split':[10],
         # 'max_features':['log2','sqrt',None],
          'max_features':['sqrt']
         }
clf = GridSearchCV(model_GB,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_GB_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('GB.xlsx')

In [ ]:
best_RF = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_GB_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_RF.loc[i,'acc_score_GB'] = acc_score
    #print(acc_score)
best_RF

In [ ]:
# model_DT调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
# 调参：https://blog.csdn.net/VariableX/article/details/107188730
#for i in range(2,50,2):
#x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=20)    
params = {'criterion': ['entropy','gini'],
          # 'criterion': ['gini'],
         'splitter': ['best','random'],
          # 'splitter': ['random'],
         # 'max_features': [None],
          'max_features': [None,'log2','sqrt','auto'],
          'max_depth':[3,5,8,10,20,30,50,None],
         # 'max_depth':[None],
         'min_samples_leaf':[1,2,5,10],
         'min_samples_split':[1,2,5,10,15,100],
          'class_weight':[None,'balanced'],
         }
clf = GridSearchCV(model_DT,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_DT_best = clf.best_estimator_
print("random_state= ",20)
print(clf.best_params_)
print(clf.best_score_)

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('DT.xlsx')

In [ ]:
best_DT = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_DT_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_DT.loc[i,'acc_score_DT'] = acc_score
    #print(acc_score)
best_DT

In [ ]:
# model_SVM调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
params = {'C': [0.001,0.01,0.1,1,10], 
          'degree': [3,2,1],
          'kernel': ['linear','poly''rbf','sigmoid'], 
          'max_iter': [1,2,5,10,20,50,-1], 
          'probability': [True,False], 
          'shrinking': [True,False],  
          'tol': [1e-1,1e-2,1e-3,1e-4,1e-5]
         }
clf = GridSearchCV(model_SVM,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_SVM_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('scv.xlsx')

In [ ]:
best_SVM = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_SVM_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_SVM.loc[i,'acc_score_SVM'] = acc_score
    #print(acc_score)
best_SVM

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=5)

In [ ]:
#先试一下KNN调参
params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,20,30],
          'algorithm':['auto','ball_tree','kd_tree','brute'],
          'leaf_size':[10,20,30,40,50,60,70,80,90,100,200,300],
          'weights':['uniform','distance']
          #'p':[1,2]
         }
clf = GridSearchCV(model_KNN,param_grid=params,cv=10)
clf.fit(x_train,y_train)

model_KNN_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
model_KNN_best

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('KNN.xlsx')

In [ ]:
X_nml[:,0:1]

In [ ]:
best_KNN = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_KNN_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_KNN.loc[i,'acc_score_KNN'] = acc_score
    #print(acc_score)
best_KNN

In [ ]:
# https://zhuanlan.zhihu.com/p/359884598
from sklearn.externals import joblib
（1）保存模型
joblib.dump(classifier, 'D:/desktop/my_knn.pkl')

（2）加载模型
my_knn = joblib.load('D:/desktop/my_knn.pkl')

In [ ]:
selfSample = pd.read_excel('SelfpredictAsite5_RFE6_ranked.xlsx')
selfSample

In [ ]:
X_prid = selfSample.iloc[:,3:6]
X_prid

In [ ]:
X_std_prid = StandardScaler().fit_transform(X_prid)
X_nml_prid = Normalizer().fit_transform(X_std_prid)
X_nml_prid

In [ ]:
y_pred = model_KNN_best.predict(X_nml_prid)
y_pred

In [ ]:
import shap

# https://stackoverflow.com/questions/62211302/obtaining-the-shap-values-for-a-prediction-made-with-knn
explainer = shap.KernelExplainer(model_KNN_best.predict_proba,x_test) #这里的model在准备工作中已经完成建模，模型名称就是model
shap_values = explainer.shap_values(X_selfA_nml) #这里的test_data是我的测试集，predictors是X_train的变量

In [ ]:
shap_values

In [ ]:
data31 = pd.read_excel('SelfpredictAsiteRFE6_ranked5.xlsx')
data31

In [ ]:
X_selfA = data31.iloc[:,1:4]
Y_selfA = data31.iloc[:,-2:-1]
X_selfA

In [ ]:
X_selfA_std = StandardScaler().fit_transform(X_selfA)
X_selfA_nml = Normalizer().fit_transform(X_selfA_std)
X_selfA_nml

In [ ]:
a = pd.DataFrame(X_selfA_nml)
a.to_excel('a.xlsx')

In [ ]:
#分层保存shap_values
a = pd.DataFrame(shap_values[2])
a.to_excel('Figure19_selfAsite_SHAPvalues2.xlsx')

In [ ]:
plt.figure(dpi=100)
plt.rcParams['Figure.figsize'] = (8.0,6.0)

cols = X.columns.tolist()
shap.summary_plot(shap_values[2],x_test,feature_names=cols,show = False)

#保存图片
plt.savefig('Figure16_x_test_ShapValues2.png',dpi=1080,bbox_inches='tight',transparent=True)

In [ ]:
shap_values[0]

In [ ]:
from IPython.core.display import HTML
HTML('<a href="http://example.com">link</a>')

In [ ]:
X.columns

In [ ]:
#更改列标题
list(X)
#X.columns
x_test_df = pd.DataFrame(x_test)
x_test_df.columns = list(X)
x_test_df

In [ ]:
force_plot = shap.force_plot(explainer.expected_value[2],shap_values[2][38,:],
                             x_test_df.iloc[38,:],show = False
                            )
shap_html = f"{shap.getjs()}{force_plot.html()}"
HTML(shap_html)
#imgkit.(force_plot.html(), 'out.png')
#plt.savefig('Figure17_2_ShapValues0_singleSampleIndex35.png',dpi=1080,bbox_inches='tight',transparent=True)

In [ ]:
explainer.expected_value[0]

In [ ]:
#查看整个测试集的Force Plot
shap.force_plot(explainer.expected_value[2], 
                shap_values[2][:38,:], x_test_df.iloc[:38,:])

In [ ]:
list(X)

In [ ]:
x_test_df.to_excel('x_test_nml.xlsx')

In [ ]:
plt.figure(dpi=200)
plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
shap.dependence_plot('SlogP_VSA2', shap_values[2], x_test_df, display_features=x_test_df,show = False)

In [ ]:
X11,y11 = shap.datasets.adult()
X11

In [ ]:
X_display,y_display = shap.datasets.adult(display=True)
X_display

In [ ]:
# model_MLP调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
params = {#'hidden_layer_sizes':[1,2,5,10,20,50,100,200,500,1000,2000],
          'hidden_layer_sizes':[1000],
          'activation':['relu'],
          #'activation':['identity','logistic','tanh','relu'],         
         # 'solver':['adam','sgd','lbfgs'],
          'solver':['adam'],
          #'alpha':[0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1],
          'alpha':[0.0005],
         # 'learning_rate':['constant','invscaling','adaptive'],
          'learning_rate':['constant'],
         # 'max_iter':[1,10,100,200,500,1000,2000],
          'max_iter':[1000],
          #'shuffle':[True,False],
          'shuffle':[False],
          'tol': [1e-1,1e-2,1e-3,1e-4,1e-5],
         }
clf = GridSearchCV(model_MLP,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_MLP_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('AB.xlsx')

In [ ]:
best_AB = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_AB_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_AB.loc[i,'acc_score_AB'] = acc_score
    #print(acc_score)
best_AB

In [ ]:
# model_LR调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
params = {'C':[0.0001, 1, 100, 1000],
          'max_iter':[1, 10, 100, 500],
          'class_weight':['balanced', None],
          'solver':['liblinear','sag','lbfgs','newton-cg']
         }
clf = GridSearchCV(model_LR,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_LR_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
# model_SGD调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
params = {#'loss': ['hinge','log','modified_huber','squared_hinge','perceptron','huber','epsilon_insensitive','squared_epsilon_insensitive'], 
          'loss': ['perceptron'],
         # 'penalty': ['l1','l2','elasticnet'],
          'penalty': ['elasticnet'],
         # 'alpha': [1e-6,1e-5,1e-4,1e-3,1e-2,0.1], 
         'alpha': [1e-4], 
          #'l1_ratio': [0.05],
        # 'fit_intercept': [True,False], 
        'fit_intercept': [True],
         # 'max_iter': [1,2,5,10,50,100],
         'max_iter': [50],
         # 'tol': [1e-4,1e-3,1e-2,1e-1,None],
        'tol': [0.1],
        #  'shuffle': [True,False],
        'shuffle': [True],
         #'learning_rate':['constant','optimal','invscaling']
          'epsilon': [0,0.0001,0.001,0.01,0.1,1,2,5],
         }
clf = GridSearchCV(model_SGD,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_SGD_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
# model_AB调参； 不用一步一步算了；  https://zhuanlan.zhihu.com/p/55438631
params = {'n_estimators': [1,2,5,10,20,50,100,200], 
          'learning_rate': [0,1,2,5,10,20,50,100,200],
          'algorithm': ['SAMME','SAMME.R'], 
         }
clf = GridSearchCV(model_AB,param_grid=params,cv=5,scoring='accuracy')
clf.fit(x_train,y_train)
model_AB_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
data7 = pd.read_excel('Figure8_AllBestModelBox_DropMinMax.xlsx')

In [ ]:
# best_RF_feature箱型图绘制
# https://blog.csdn.net/weixin_40683253/article/details/87857194
boxprops={'color':'black','facecolor':'#84c1bc','linewidth':'1'}
medianprops = dict(linewidth=1.5, color='#3d85ab')
labels = '1','2','3','4','5','6','7','8','9'
font = {'family':'Arial',
        'size':'16'
       }
    
box_1,box_2,box_3,box_4,box_5,box_6,box_7,box_8,box_9 = data7['acc_score_RF'],data7['acc_score_GB'],data7['acc_score_DT'],data7['acc_score_SVM'],data7['acc_score_KNN'],data7['acc_score_MLP'],data7['acc_score_LR'],data7['acc_score_SGD'],data7['acc_score_AB'],
a = [box_1,box_2,box_3,box_4,box_5,box_6,box_7,box_8,box_9] 

plt.figure(dpi=100)
#plt.minorticks_on()
plt.tick_params(top=False,bottom=True,left=True,right=False)
#ax = plt.gca()
#ax.axes.xaxis.set_visible(True)
plt.boxplot(a,labels=labels,patch_artist=True,showfliers=False,boxprops=boxprops,medianprops=medianprops)
#plt.grid(True)
#plt.style.use('seaborn-white')
plt.xlabel('Number of Features',fontdict=font)
plt.ylabel('Accurancy Score',fontdict=font)
plt.xticks(fontsize=12,color='black')
plt.yticks(fontsize=12)

plt.tick_params(direction='in')
#plt.rcParams['xtick.direction'] = 'in' #刻度朝内
#plt.rcParams['ytick.direction'] = 'in' #刻度朝内
#plt.grid(linestyle="--",alpha=0.3)
plt.ylim(0.4,1) #定义y取值范围
plt.show()
#plt.savefig("bestRF_Nfeatures.jpg",dpi=1600,bbox_inches='tight')

In [ ]:
model_KNN_best

In [ ]:
data8 = pd.read_excel('V8_RFE6_ranked.xlsx')
data8.head()

In [ ]:
X = data8.iloc[:,3:6]
Y = data8.iloc[:,2:3]
X

In [ ]:
X_std = StandardScaler().fit_transform(X)
X_nml = Normalizer().fit_transform(X_std)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=10)

In [ ]:
#先试一下KNN调参
params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,20,30],
          'algorithm':['auto','ball_tree','kd_tree','brute'],
          'leaf_size':[10,20,30,40,50,60,70,80,90,100,200,300],
          'weights':['uniform','distance']
          #'p':[1,2]
         }
clf = GridSearchCV(model_KNN,param_grid=params,cv=10)
clf.fit(x_train,y_train)

model_KNN_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
best_KNN = pd.DataFrame()
for i in range(1,51):
    x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=i)
    y_pred = model_KNN_best.predict(x_test)
    acc_score = accuracy_score(y_test,y_pred)
    best_KNN.loc[i,'acc_score_KNN'] = acc_score
    #print(acc_score)
best_KNN

In [ ]:
BestModel_params = pd.DataFrame.from_dict(clf.best_params_,orient='index')
BestModel_params.to_excel('AB.xlsx')

In [ ]:
y_test

In [ ]:
#前41个特征的相关性热图
X_corr = round(X.corr().abs(),2)

plt.figure(dpi=200)
sns.set(font_scale=1.2)
plt.rc('font',family='Arial')
sns.heatmap(X_corr,cmap="GnBu",
            annot=True,annot_kws={'size':14},
           vmax=1,
           linewidths=1)
plt.xticks(rotation=90)
plt.savefig('Figure7_heatmapPersen6.png',dpi=2160,bbox_inches='tight')
#plt.show()

In [ ]:
#混淆矩阵
from sklearn.metrics import confusion_matrix

y_pred = model_KNN_best.predict(x_test)
CM = confusion_matrix(y_test,y_pred,labels=[0,1,2])
plt.figure(dpi=200)



plt.rc('font',family='Arial')
sns.set(font_scale=1.8)
sns.heatmap(CM,cmap="GnBu",annot=True,annot_kws={'size':16},linewidths=2)
plt.ylabel('True Dimensionality')
plt.xlabel('Predicted Dimensionality')
plt.savefig('Figure11_confusionMatrix.png',bbox_inches='tight',dpi=1600)

In [ ]:
df1 = pd.read_excel('V12_6FeatureKDEViolin_nml.xlsx')
df1

In [ ]:
df1['ATSC1pe']

In [ ]:
X = df1.iloc[:,2:]
Y = df1.iloc[:,1:2]
X

In [ ]:
X.plot(kind='hist')

In [ ]:
df1['ATSC1pe'].hist(bins=10)

In [ ]:
#plt.figure(dpi=200)
sns.pairplot(df1,hue='Dimensionality',diag_kind='kde',
             height=3,
             #markers=["o", "s", "D"],
             palette="tab10",
             corner=False,
             #diag_kind="hist"
            )
#hue='Dimensionality',palette="tab10"


#plt.savefig('pairplot.png',dpi=2160,bbox_inches='tight',transparent=True)

In [ ]:
sns.set_style('darkgrid')

In [ ]:
#plt.figure(dpi=200)
g = sns.PairGrid(df1,hue='Dimensionality',palette="tab10",)
g.map_lower(sns.scatterplot,s=80,alpha=0.5,ec='black')
g.map_diag(sns.kdeplot,fill=True)
g.map_upper(sns.scatterplot,s=80,alpha=0.5,ec='black')
g.add_legend()
#g.axes().tick_params(direction='in')
#g.map_upper(sns.kdeplot,n_levels=6)


#plt.savefig('pairplot.png',dpi=2160,bbox_inches='tight',transparent=True)

In [ ]:
df1['Dimensionality'].unique()

In [ ]:
ATSC1pe 	MATS2c 	SlogP_VSA2 	ATSC3c 	GATS2c 	AATSC2c

In [ ]:
# KDE图
plt.figure(dpi=100)
plt.rcParams['figure.figsize'] = (8.0,6.0)

D2 = df1.loc[df1.Dimensionality==2] #取出所有2D数据
D1 = df1.loc[df1.Dimensionality==1] #取出所有1D数据
D0 = df1.loc[df1.Dimensionality==0] #取出所有0D数据
#插入kde图
sns.kdeplot(D1.MATS2c,D1.AATSC2c,cmap='Blues',shade=True,shade_lowest=False,n_levels=8,alpha=1)
sns.kdeplot(D0.MATS2c,D0.AATSC2c,cmap='Greens',shade=True,shade_lowest=False,n_levels=8,alpha=1)
sns.kdeplot(D2.MATS2c,D2.AATSC2c,cmap='Reds',shade=True,shade_lowest=False,n_levels=8,alpha=1)
#sns.kdeplot(df1['AATSC2c'],df1['MATS2c'],hue=df1['Dimensionality'],palette="Set2",shade=True,shade_lowest=False,n_levels=8)

# 获取/调整边框 https://www.cxyzjd.com/article/qq_40481843/106231257 
ax1=plt.gca()  
bwith = 2
ax1.spines['bottom'].set_linewidth(bwith)
ax1.spines['left'].set_linewidth(bwith)
ax1.spines['top'].set_linewidth(bwith)
ax1.spines['right'].set_linewidth(bwith)

#调整x/y取值范围
plt.xlim(-1.6,1.6)
plt.ylim(-1.6,1.6)

#刻度和方向
plt.rcParams["ytick.left"] = True 
plt.rcParams["xtick.bottom"] = True
plt.rcParams["xtick.top"] = False 
plt.rcParams["ytick.right"] = False
plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.tick_params(axis='both',width=2)
#刻度间隔
x_major_locator=MultipleLocator(0.6)
y_major_locator=MultipleLocator(0.8)
ax1.xaxis.set_major_locator(x_major_locator)
ax1.yaxis.set_major_locator(y_major_locator)
#标签大小
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

#标题和大小
ax = plt.subplot()
ax.set_xlabel(xlabel='MATS2c',fontsize=22)
ax.set_ylabel(ylabel='AATSC2c',fontsize=22)

#保存图片
plt.savefig('Figure12_13_kde_MATS2c_AATSC2c.png',dpi=1080,bbox_inches='tight',transparent=True)

In [ ]:
ATSC1pe 	MATS2c 	SlogP_VSA2 	ATSC3c 	GATS2c 	AATSC2c

In [ ]:
df2 = pd.read_excel('V12_6FeatureKDEViolin_nml.xlsx')
df2

In [ ]:
X2 = df2.iloc[:,2:]
Y2 = df2.iloc[:,1:2]
Y2

In [ ]:
ATSC1pe 	MATS2c 	SlogP_VSA2 	ATSC3c 	GATS2c 	AATSC2c

In [ ]:
# 小提琴图

plt.figure(dpi=100)
plt.rcParams['figure.figsize'] = (8.0,6.0)

#插入小提琴图
sns.violinplot(data=df2,x='Dimensionality',y='SlogP_VSA2',cut=5,zorder=300,palette='GnBu')
#sns.set_style('white')



# 获取/调整边框 https://www.cxyzjd.com/article/qq_40481843/106231257 
ax1=plt.gca()  
bwith = 2
ax1.spines['bottom'].set_linewidth(bwith)
ax1.spines['left'].set_linewidth(bwith)
ax1.spines['top'].set_linewidth(bwith)
ax1.spines['right'].set_linewidth(bwith)

#刻度和方向
plt.rcParams["ytick.left"] = True 
plt.rcParams["xtick.bottom"] = True
plt.rcParams["xtick.top"] = False 
plt.rcParams["ytick.right"] = False
plt.rcParams['xtick.direction'] = 'in' 
plt.rcParams['ytick.direction'] = 'in' 
plt.tick_params(axis='both',width=2)
#单位
x_major_locator=MultipleLocator(1)
y_major_locator=MultipleLocator(0.8)
ax1.xaxis.set_major_locator(x_major_locator)
ax1.yaxis.set_major_locator(y_major_locator)

#定义背景
#plt.grid(color='black',linestyle='--',linewidth=1,alpha=0.1,zorder=0) 

#标签大小
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)

#标题和大小
ax = plt.subplot()
ax.set_xlabel(xlabel='Dimensionality',fontsize=28)
ax.set_ylabel(ylabel='SlogP_VSA2',fontsize=28)


#保存图片
plt.savefig('Figure13_3_Violin_SlogP_VSA2.png',dpi=1080,bbox_inches='tight',transparent=True)

In [ ]:
data9 = pd.read_excel('V8_RFE6_ranked.xlsx')
data9

In [ ]:
X = data9.iloc[:,3:6]
Y = data9.iloc[:,2:3]
X

In [ ]:
X_std = StandardScaler().fit_transform(X)
X_nml = Normalizer().fit_transform(X_std)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X_nml,Y,test_size=0.2,random_state=5)

In [ ]:
x_train

In [ ]:
#先试一下KNN调参
params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,20,30],
          'algorithm':['auto','ball_tree','kd_tree','brute'],
          'leaf_size':[10,20,30,40,50,60,70,80,90,100,200,300],
          'weights':['uniform','distance']
          #'p':[1,2]
         }
clf = GridSearchCV(model_KNN,param_grid=params,cv=10)
clf.fit(x_train,y_train)

model_KNN_best = clf.best_estimator_
print(clf.best_params_)
clf.best_score_

In [ ]:
data16 = pd.read_excel('SelfpredictAsite_SHAPranked5.xlsx')
data16

In [ ]:
X16 = data16.iloc[:,3:5]
Y16 = data16.iloc[:,1:2]
Y16

In [ ]:
X16_std = StandardScaler().fit_transform(X16)
X16_nml = Normalizer().fit_transform(X16_std)
X16_nml

In [ ]:
model_KNN2 = KNeighborsClassifier(leaf_size=10, n_neighbors=20,weights='distance',algorithm='auto')
model_KNN2.fit(x_train,y_train)
print(model_KNN2.predict_proba(x_train))

In [ ]:
y_train.to_excel("d.xlsx")

In [ ]:
a = pd.DataFrame(model_KNN2.predict_proba(x_train)) 
a.to_excel("c.xlsx")

In [ ]:
# https://cloud.tencent.com/developer/article/1725049
#model_KNN2 = KNeighborsClassifier(leaf_size=10, n_neighbors=20,weights='distance',algorithm='auto')
#model_RF2 = RandomForestClassifier(criterion='gini',max_depth=10,min_samples_leaf=3,min_samples_split=2,n_estimators=10)
#model_KNN_best2 = model_KNN_best(n_neighbors=10)
model_KNN_best.fit(x_train,y_train)
print(model_KNN_best.predict_proba(x_test))

In [ ]:
y_train.to_excel('b.xlsx')

In [ ]:
best_KNN_predict_proba = pd.DataFrame(model_KNN_best.predict_proba(x_test))
best_KNN_predict_proba.to_excel('d.xlsx')

In [ ]:
y_test

In [ ]:
# http://www.ecmlpkdd2018.org/wp-content/uploads/2018/09/749.pdf

In [ ]:
import pysubgroup as ps

In [ ]:
data10 = pd.read_excel('V8_RFE6_ranked.xlsx')
data10.head(5)

In [ ]:
X = data10.iloc[:,3:6]
Y = data10.iloc[:,2:3]
X

In [ ]:
target = ps.BinaryTarget('Dimensionality',True)
# searchspace = ps.create_selectors(data10,ignore=['Unnamed: 0','序号','Dimensionality'])
searchspace = ps.create_selectors(X)
task = ps.SubgroupDiscoveryTask(
    data10,
    target,
    searchspace,
    result_set_size=5,
    depth=2,
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)


In [ ]:
result.to_dataframe()

In [ ]:
result.to_dataframe().to_excel('Table3_ARM_PySubgroup_3Features.xlsx')

In [ ]:
# https://www.geeksforgeeks.org/implementing-apriori-algorithm-in-python/
# https://www.section.io/engineering-education/apriori-algorithm-in-python/

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
data10.head()

In [ ]:
data10.columns

In [ ]:
data10.Dimensionality.unique()

In [ ]:
data10_2D = (data10[data10['Dimensionality']=='2'].groupby(['ATSC1pe','MATS2c', 'SlogP_VSA2',
       'ATSC3c', 'GATS2c', 'AATSC2c'])['MATS2c']
          .sum().unstack().reset_index().fillna(0)
          .set_index('ATSC1pe'))
data10_2D

In [ ]:
# https://www.kaggle.com/code/averkij/conditional-variational-autoencoder-and-t-sne 

In [ ]:
pwd

In [ ]:
cd ./CVAE_master/

In [ ]:
# %load cal_prop.py

from rdkit.Chem.Descriptors import ExactMolWt
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.rdMolDescriptors import CalcNumHBD
from rdkit.Chem.rdMolDescriptors import CalcNumHBA
from rdkit.Chem.rdMolDescriptors import CalcTPSA
from rdkit import Chem
from multiprocessing import Pool
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--input_filename', help='filename for smiles', type=str, default='smiles2.txt')
parser.add_argument('--output_filename', help='name of output file', type=str, default='smiles2_prop.txt')
parser.add_argument('--ncpus', help='number of cpus', type=int, default=1)
args,unknown = parser.parse_known_args()

def cal_prop(s):
    m = Chem.MolFromSmiles(s)
    if m is None : return None
    return Chem.MolToSmiles(m), ExactMolWt(m), MolLogP(m), CalcNumHBD(m), CalcNumHBA(m), CalcTPSA(m)
    #return Chem.MolToSmiles(m), ExactMolWt(m), MolLogP(m), CalcNumHBD(m), CalcNumHBA(m), CalcTPSA(m)

with open(args.input_filename) as f:
    smiles = f.read().split('\n')[:-1]
pool = Pool(8)

r = pool.map_async(cal_prop, smiles)

data = r.get()
pool.close()
pool.join()
w = open(args.output_filename, 'w')
for d in data:
    if d is None:
        continue
    w.write(d[0] + '\t' + str(d[1]) + '\t'+ str(d[2]) + '\t'+ str(d[3]) + '\n')
w.close()    



In [ ]:
import sys
sys.version